In [ ]:
import os
import numpy as np

def load_and_separate_npz_data(root_dir):
    # Initialize a dictionary to hold separated data for each tag
    separated_data = {
        'episode_reward': [],
        'loss': [],
        'runtime': [],
        'params_gradients': [],
        'input_params_gradients': []
    }

    # Walk through all directories and subdirectories in the root directory
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.npz'):
                # Construct full file path
                file_path = os.path.join(subdir, file)
                
                # Load the .npz file if it exists
                if os.path.exists(file_path):
                    data = np.load(file_path, allow_pickle=True)
                    
                    # Append the data from this file to the respective lists in the separated data dictionary
                    for key in separated_data:
                        if key in data:
                            # Append each file's data as a new list within the main list
                            separated_data[key].append(data[key].tolist())

    return separated_data

# Usage example:
root_directory = os.getcwd()  # Use the current working directory or specify a path
separated_data = load_and_separate_npz_data(root_directory)

# Now you have all the data separated into lists of lists per tag
print("Separated Episode Rewards:", separated_data['episode_reward'])
print("Separated Losses:", separated_data['loss'])
print("Separated Runtimes:", separated_data['runtime'])
print("Separated Parameter Gradients:", separated_data['params_gradients'])
print("Separated Input Parameter Gradients:", separated_data['input_params_gradients'])

In [22]:
import pandas as pd
import numpy as np
import os

class Analysis():
    """
    """

    def __init__(self, path_to_dir):
        self.path = path_to_dir

    def load_data(self):
        separated_data = {
                'episode_reward': [],
                'loss': [],
                'runtime': [],
                'params_gradients': [],
                'input_params_gradients': []
            }

        # Walk through all directories and subdirectories in the root directory
        for subdir, dirs, files in os.walk(self.path):
            for file in files:
                if file.endswith('.npz'):
                    # Construct full file path
                    file_path = os.path.join(subdir, file)
                    
                    # Load the .npz file if it exists
                    if os.path.exists(file_path):
                        data = np.load(file_path, allow_pickle=True)
                        
                        # Append the data from this file to the respective lists in the separated data dictionary
                        for key in separated_data:
                            if key in data:
                                # Append each file's data as a new list within the main list
                                separated_data[key].append(data[key].tolist())

        self.data = separated_data
       
    def get_rewards(self):
        return self.data["episode_reward"]
    
    def get_loss(self):
        return self.data["loss"]
    
    def get_runtime(self):
        return self.data["runtime"]
    
    def get_gradients(self):
        return self.data["params_gradients"] 
    
    def get_input_gradients(self):
        return self.data["input_params_gradients"]

    def get_moving_average(self, window_size = 10):
        rewards = self.get_rewards()
        moving_averages = []
        for reward in rewards:
            moving_averages.append(pd.Series(reward).rolling(window_size).mean())
        return moving_averages
    
    def calculate_mean_variance_gradients(self, return_max = False, return_min = False):
        gradients = self.get_gradients()
        min_length = min([len(gradients[i]) for i in range(len(gradients))])

        gradients = [gradients[i][:min_length] for i in range(len(gradients))]

        def flatten_gradients(gradients):
            for i in range(len(gradients)):
                for j in range(len(gradients[i])):
                    gradients[i][j] = np.concatenate([lista.flatten() for lista in gradients[i][j]], axis = 0)

        flatten_gradients(gradients)

        gradients_array = np.array(gradients)

        magnitudes_gradients = np.linalg.norm(gradients_array, axis = 2)

        mean_magnitudes_gradients = np.mean(magnitudes_gradients, axis = 0)

        std_magnitudes_gradients = np.var(magnitudes_gradients, axis = 0)

        max_magnitudes_gradients = np.max(magnitudes_gradients, axis = 0)

        min_magnitudes_gradients = np.min(magnitudes_gradients, axis = 0)

        max_index = np.argmax(gradients_array, axis = 2)

        min_index = np.argmin(gradients_array, axis = 2)

        if return_max and return_min:
            return mean_magnitudes_gradients, std_magnitudes_gradients, max_magnitudes_gradients, max_index, min_magnitudes_gradients, min_index
        elif return_max:
            return mean_magnitudes_gradients, std_magnitudes_gradients, max_magnitudes_gradients, max_index
        elif return_min:
            return mean_magnitudes_gradients, std_magnitudes_gradients, min_magnitudes_gradients, min_index
        else:
            return mean_magnitudes_gradients, std_magnitudes_gradients
        
    def calculate_mean_variance_input_gradients(self, return_max = False, return_min = False):
        gradients = self.get_input_gradients()
        min_length = min([len(gradients[i]) for i in range(len(gradients))])

        gradients = [gradients[i][:min_length] for i in range(len(gradients))]

        def flatten_gradients(gradients):
            for i in range(len(gradients)):
                for j in range(len(gradients[i])):
                    gradients[i][j] = np.concatenate([lista.flatten() for lista in gradients[i][j]], axis = 0)

        flatten_gradients(gradients)

        gradients_array = np.array(gradients)

        magnitudes_gradients = np.linalg.norm(gradients_array, axis = 2)

        mean_magnitudes_gradients = np.mean(magnitudes_gradients, axis = 0)

        std_magnitudes_gradients = np.var(magnitudes_gradients, axis = 0)

        max_magnitudes_gradients = np.max(magnitudes_gradients, axis = 0)

        min_magnitudes_gradients = np.min(magnitudes_gradients, axis = 0)

        max_index = np.argmax(gradients_array, axis = 2)

        min_index = np.argmin(gradients_array, axis = 2)

        if return_max and return_min:
            return mean_magnitudes_gradients, std_magnitudes_gradients, max_magnitudes_gradients, max_index, min_magnitudes_gradients, min_index
        elif return_max:
            return mean_magnitudes_gradients, std_magnitudes_gradients, max_magnitudes_gradients, max_index
        elif return_min:
            return mean_magnitudes_gradients, std_magnitudes_gradients, min_magnitudes_gradients, min_index
        else:
            return mean_magnitudes_gradients, std_magnitudes_gradients
        

In [ ]:
def average_agents(data):
    moving_averages = []
    for agent in data:
        moving_averages.append(moving_average(agent))

    all_agents_average = [np.mean(returns) for returns in moving_averages]